In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Imports helper functions
from kaggle_environments.envs.halite.helpers import *

# Returns best direction to move from one position (fromPos) to another (toPos)
# Example: If I'm at pos 0 and want to get to pos 55, which direction should I choose?
def getDirTo(fromPos, toPos, size):
    x1,x2,x,y1,y2,y = 0,0,0,0,0,0
    if fromPos[1] < 10:
        y1 = fromPos[1]
    if fromPos[1] > 10:
        y1 = size - fromPos[1]
    if toPos[1] < 10:
        y2 = toPos[1]
    if toPos[1] > 10:
        y2 = size - toPos[1]
    if abs(fromPos[1]-toPos[1])<=(y1+y2) and abs(fromPos[1]-toPos[1])!=0 and (y1+y2)!=0:
        if (fromPos[1]<toPos[1]):
            return ShipAction.NORTH
        else:
            return ShipAction.SOUTH
    if (y1+y2)<abs(fromPos[1]-toPos[1]):
        if fromPos[1] < 10:
            return ShipAction.SOUTH
        if fromPos[1] >= 10:
            return ShipAction.NORTH
        
    if fromPos[0] < 10:
        x1 = fromPos[0]
    if fromPos[0] > 10:
        x1 = size - fromPos[0]
    if toPos[0] < 10:
        x2 = toPos[0]
    if toPos[0] > 10:
        x2 = size - toPos[0]
    if abs(fromPos[0]-toPos[0])<=(x1+x2) and abs(fromPos[0]-toPos[0])!=0 and (x1+x2)!=0:
        if (fromPos[0]<toPos[0]):
            return ShipAction.EAST
        else:
            return ShipAction.WEST
    if (x1+x2)<abs(fromPos[0]-toPos[0]):
        if fromPos[0] < 10:
            return ShipAction.WEST
        if fromPos[0] >= 10:
            return ShipAction.EAST

def getDistTo(fromPos, toPos, size):
    x1,x2,x,y1,y2,y = 0,0,0,0,0,0
    if fromPos[0] <= 10:
        x1 = fromPos[0]
    if fromPos[0] > 10:
        x1 = size - fromPos[0]
    if toPos[0] <= 10:
        x2 = toPos[0]
    if toPos[0] > 10:
        x2 = size - toPos[0]
    x = min(abs(fromPos[0]-toPos[0]),x1+x2)
    if fromPos[1] <= 10:
        y1 = fromPos[1]
    if fromPos[1] > 10:
        y1 = size - fromPos[1]
    if toPos[1] <= 10:
        y2 = toPos[0]
    if toPos[1] > 10:
        y2 = size - toPos[1]
    y = min(abs(fromPos[1]-toPos[1]),y1+y2)
    distance = x+y
    return distance

def shipAboveShipyard(cell, me):
    if cell.ship == None:
        return True
    return False

def getSafestAndQuickestPath(priorityCell, safeCells, size):
    bestCell = None
    lowestDist = 100
    for cell in safeCells:
        dist = getDistTo(cell.position, priorityCell.position, size)
        if dist < lowestDist:
            lowestDist = dist
            bestCell = cell
    return bestCell

def enemyShipNear(cell, me):
    surroundingCells = [cell.north, cell.east, cell.south, cell.west]
    for cell in surroundingCells:
        if cell.ship != None and cell.ship.player != me:
            return True
    return False

def nearestShipyard(ship, shipyards, size):
    steps = 100
    nearest_shipyard = None
    for shipyard in shipyards:
        distance = getDistTo(ship.position, shipyard.position,size)
        if (distance < steps):
            steps = distance
            nearest_shipyard = shipyard
    return(nearest_shipyard.cell, steps)

def stateOfCell(cell):
    if cell.shipyard == None and cell.ship == None:
        return True #Safe
    return False #Not Safe

def getShipPositions(board, me):
    occupiedCells = {}
    for x in board.cells:
        if board.cells[x].ship != None and board.cells[x].ship.player == me:
            occupiedCells[board.cells[x].ship.id] = board.cells[x]
    return occupiedCells

def cellWithHighestHalite(safeCells, oneStepAheadPos):
    halite = 10
    haliteCell = None
    for cell in safeCells:
        if cell.halite > halite and cell.position not in oneStepAheadPos:
            halite = cell.halite
            haliteCell = cell
    return(haliteCell)

def getNearbyHighestHaliteCell(board, ship, size, occupiedCells, oneStepAheadPos):
    highestHalite = 10
    distToHighestHalite = 100
    cell = None
    for x in board.cells:
        if board.cells[x].position in oneStepAheadPos or board.cells[x].position in goToPos.values() or board.cells[x].ship != None or board.cells[x].shipyard != None:
            continue
        pos = board.cells[x].position
        hal = board.cells[x].halite
        dist = getDistTo(ship.position, pos, size)
        if(hal > highestHalite):
            highestHalite = hal
            highestHaliteCellPosition = pos
            distToHighestHalite = dist
            cell = board.cells[x]
        if (hal == highestHalite):
             if (dist <= distToHighestHalite):
                 highestHalite = hal
                 highestHaliteCellPosition = pos
                 distToHighestHalite = dist
                 cell = board.cells[x]
    return cell

def getEnemies(ship, me):
    haliteNextTurn = ship.halite + (0.25 * ship.cell.halite)
    enemies = []
    surroundingCells = [ship.cell.north, ship.cell.east, ship.cell.south, ship.cell.west, ship.cell.north.east, ship.cell.east.south, ship.cell.south.west, ship.cell.west.north]
    for cell in surroundingCells:
        if cell.ship != None and cell.ship.player != me and cell.ship.halite <= haliteNextTurn:
            enemies.append(cell.ship)
    if len(enemies) == 0:
        return False, None
    else:
        return True, enemies
    
def getSafeRunAwayCells(ship, enemyShips, oneStepAheadPos):
    haliteNextTurn = ship.halite + (0.25 * ship.cell.halite)
    safeCells = []
    for enemyShip in enemyShips:
        if ship.cell.north.shipyard != None and ship.cell.north.ship == None and ship.cell.north.shipyard not in unsafeShipyards and ship.cell.north.shipyard.player == ship.player and ship.cell.north.position not in oneStepAheadPos:
            safeCells.append(ship.cell.north)
        
        if ship.cell.east.shipyard != None and ship.cell.east.ship == None and ship.cell.east.shipyard not in unsafeShipyards and ship.cell.east.shipyard.player == ship.player and ship.cell.east.position not in oneStepAheadPos:
            safeCells.append(ship.cell.east)
            
        if ship.cell.south.shipyard != None and ship.cell.south.ship == None and ship.cell.south.shipyard not in unsafeShipyards and ship.cell.south.shipyard.player == ship.player and ship.cell.south.position not in oneStepAheadPos:
            safeCells.append(ship.cell.south)
            
        if ship.cell.west.shipyard != None and ship.cell.west.ship == None and ship.cell.west.shipyard not in unsafeShipyards and ship.cell.west.shipyard.player == ship.player and ship.cell.west.position not in oneStepAheadPos:
            safeCells.append(ship.cell.west)
        
        if stateOfCell(ship.cell.north) and ship.cell.north.position not in oneStepAheadPos:
            surroundingCells = [ship.cell.north.west, ship.cell.north.north, ship.cell.north.east]
            threat = False
            for cell in surroundingCells:
                if cell.ship == None and cell.shipyard == None:
                    continue
                elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                    threat = True
                elif cell.ship != None and cell.ship.player != ship.player:
                    if cell.ship.halite < haliteNextTurn:
                        threat = True
            if threat == False:
                safeCells.append(ship.cell.north)
                
        elif ship.cell.north.ship != None and ship.cell.north.ship.player != ship.player and ship.cell.north.position not in oneStepAheadPos:
            if ship.cell.north.ship.halite >  haliteNextTurn:
                surroundingCells = [ship.cell.north.west, ship.cell.north.north, ship.cell.north.east]
                threat = False
                for cell in surroundingCells:
                    if cell.ship == None and cell.shipyard == None:
                        continue
                    elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                        threat = True
                    elif cell.ship != None and cell.ship.player != ship.player:
                        if cell.ship.halite < haliteNextTurn:
                            threat = True
                if threat == False:
                    safeCells.append(ship.cell.north)
                    
        if stateOfCell(ship.cell.south) and ship.cell.south.position not in oneStepAheadPos:
            surroundingCells = [ship.cell.south.east, ship.cell.south.south, ship.cell.south.west]
            threat = False
            for cell in surroundingCells:
                if cell.ship == None and cell.shipyard == None:
                    continue
                elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                    threat = True
                elif cell.ship != None and cell.ship.player != ship.player:
                    if cell.ship.halite < haliteNextTurn:
                        threat = True
            if threat == False:
                safeCells.append(ship.cell.south)
                
        elif ship.cell.south.ship != None and ship.cell.south.ship.player != ship.player and ship.cell.south.position not in oneStepAheadPos:
            if ship.cell.south.ship.halite >  haliteNextTurn:
                surroundingCells = [ship.cell.south.east, ship.cell.south.south, ship.cell.south.west]
                threat = False
                for cell in surroundingCells:
                    if cell.ship == None and cell.shipyard == None:
                        continue
                    elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                        threat = True
                    elif cell.ship != None and cell.ship.player != ship.player:
                        if cell.ship.halite < haliteNextTurn:
                            threat = True
                if threat == False:
                    safeCells.append(ship.cell.south)
                
        if stateOfCell(ship.cell.east) and ship.cell.east.position not in oneStepAheadPos:
            surroundingCells = [ship.cell.east.north, ship.cell.east.east, ship.cell.east.south]
            threat = False
            for cell in surroundingCells:
                if cell.ship == None and cell.shipyard == None:
                    continue
                elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                    threat = True
                elif cell.ship != None and cell.ship.player != ship.player:
                    if cell.ship.halite < haliteNextTurn:
                        threat = True
            if threat == False:
                safeCells.append(ship.cell.east)
                
        elif ship.cell.east.ship != None and ship.cell.east.ship.player != ship.player and ship.cell.east.position not in oneStepAheadPos:
            if ship.cell.east.ship.halite >  haliteNextTurn:
                surroundingCells = [ship.cell.east.north, ship.cell.east.east, ship.cell.east.south]
                threat = False
                for cell in surroundingCells:
                    if cell.ship == None and cell.shipyard == None:
                        continue
                    elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                        threat = True
                    elif cell.ship != None and cell.ship.player != ship.player:
                        if cell.ship.halite < haliteNextTurn:
                            threat = True
                if threat == False:
                    safeCells.append(ship.cell.east)
                    
        if stateOfCell(ship.cell.west) and ship.cell.west.position not in oneStepAheadPos:
            surroundingCells = [ship.cell.west.south, ship.cell.west.west, ship.cell.west.north]
            threat = False
            for cell in surroundingCells:
                if cell.ship == None and cell.shipyard == None:
                    continue
                elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                    threat = True
                elif cell.ship != None and cell.ship.player != ship.player:
                    if cell.ship.halite < haliteNextTurn:
                        threat = True
            if threat == False:
                safeCells.append(ship.cell.west)
                
        elif ship.cell.west.ship != None and ship.cell.west.ship.player != ship.player and ship.cell.west.position not in oneStepAheadPos:
            if ship.cell.west.ship.halite >  haliteNextTurn:
                surroundingCells = [ship.cell.west.south, ship.cell.west.west, ship.cell.west.north]
                threat = False
                for cell in surroundingCells:
                    if cell.ship == None and cell.shipyard == None:
                        continue
                    elif cell.shipyard != None and cell.shipyard.player != ship.player and cell.ship != None:
                        threat = True
                    elif cell.ship != None and cell.ship.player != ship.player:
                        if cell.ship.halite < haliteNextTurn:
                            threat = True
                if threat == False:
                    safeCells.append(ship.cell.west)
                    
    return safeCells

def getSafeCells(cell, me, oneStepAheadPos):
    haliteNextTurn = cell.ship.halite + (0.25 * cell.halite)
    safeCells = []
    previousCell = None
    immediateCells = {cell.north:cell.north.north, cell.east:cell.east.east, cell.south:cell.south.south, cell.west:cell.west.west}
    
    if cell.ship.id in oneStepBehindCells.keys():
        previousCell = oneStepBehindCells[cell.ship.id]
        
    for immediateCell in immediateCells.keys():
        
        if (previousCell and immediateCell.position == previousCell.position) or immediateCell.position in oneStepAheadPos:
            continue
                
        if immediateCell.shipyard == None and immediateCell.ship == None:
            if immediateCells[immediateCell].ship == None:
                safeCells.append(immediateCell)
            elif immediateCells[immediateCell].ship != None:
                if immediateCells[immediateCell].ship.halite > haliteNextTurn or immediateCells[immediateCell].ship.player == cell.ship.player:
                    safeCells.append(immediateCell)
                    
        elif immediateCell.ship != None and immediateCell.ship.player != cell.ship.player:
            if immediateCells[immediateCell].ship == None:
                safeCells.append(immediateCell)
            elif immediateCells[immediateCell].ship != None:
                if immediateCells[immediateCell].ship.halite > haliteNextTurn or immediateCells[immediateCell].ship.player == cell.ship.player:
                    safeCells.append(immediateCell)
                
        elif immediateCell.shipyard != None:
            if immediateCell.ship == None and immediateCell.shipyard.player == me and immediateCell.shipyard not in unsafeShipyards:
                safeCells.append(immediateCell)
        
    return(safeCells)

# Directions a ship can move
directions = [ShipAction.NORTH, ShipAction.EAST, ShipAction.SOUTH, ShipAction.WEST]

# Will keep track of whether a ship is collecting halite or carrying cargo to a shipyard
goToCell, goToPos = {}, {}
oneStepBehindCells, unsafeShipyards = [], []
# Returns the commands we send to our ships and shipyards
def agent(obs, config):
    global oneStepBehindCells
    global goToCell
    global goToPos
    global unsafeShipyards
    global halll
    halll=0
    oneStepAheadPos, unsafeShipyards = [], []
    spawnLimit = 12
    spawnCost = config.spawnCost #500
    convertCost = config.convertCost #500
    convertThreshold = spawnCost + convertCost*2
    size = config.size #21
    board = Board(obs, config) #{'episodeSteps': 400, 'startingHalite': 24000, 'size': 21, 'spawnCost': 500, 'convertCost': 500, 'moveCost': 0, 'collectRate': 0.25, 'regenRate': 0.02, 'maxCellHalite': 500}
    me = board.current_player
    occupiedCells = getShipPositions(board, me)
        
    for shipyard in me.shipyards:
        for ship in me.ships: 
            halll+=ship.halite
        if(board.step<100):
            if(me.halite+halll)<1500:
                spawnLimit=12-len(me.shipyards)+1
            else:
                spawnLimit=20-len(me.shipyards)+1
        elif(board.step>=100 and board.step<270):
            if(me.halite+halll)<1500:
                spawnLimit=7-len(me.shipyards)+1
            else:
                spawnLimit=15-len(me.shipyards)+1
        else:
            if(me.halite+halll)<1500:
                spawnLimit=3-len(me.shipyards)+1
            else:
                spawnLimit=12-len(me.shipyards)+1
        if shipyard.next_action == None:
            if len(me.shipyards)>0 and me.halite >= spawnCost and len(me.ships)<spawnLimit and shipAboveShipyard(shipyard.cell, me):
                unsafeShipyards.append(shipyard)
                shipyard.next_action = ShipyardAction.SPAWN
                
    
    for ship in me.ships:
        if board.step == 398 and ship.halite>convertCost:
            ship.next_action = ShipAction.CONVERT
                
        if board.step==0:
            me.ships[0].next_action = ShipAction.CONVERT
                
        if ship.id in goToCell.keys() and ship.cell.position == goToCell[ship.id].position:
            del goToCell[ship.id]
            del goToPos[ship.id]
                
        if len(me.shipyards)==0 and ship.halite > convertCost:
            ship.next_action = ShipAction.CONVERT
                
        if(ship.next_action == None):
            safeCells = getSafeCells(ship.cell, me, oneStepAheadPos)
            enemyNear, enemies = getEnemies(ship, me)
                
            if (enemyNear == True):   
                
                if (enemies != None or len(enemies) > 0) and len(me.shipyards)>0:
                    if ship.id in goToCell.keys():
                        del goToCell[ship.id]
                        del goToPos[ship.id]
                    
                    shipyardCell, steps = nearestShipyard(ship, me.shipyards, size)
                    
                    if steps > 15:
                        ship.next_action = ShipAction.CONVERT
                        continue
                
                    safeCells = getSafeRunAwayCells(ship, enemies, oneStepAheadPos)
                    
                    if safeCells != None and len(safeCells) > 0:
                        
                        cell = getSafestAndQuickestPath(shipyardCell, safeCells, size)
                        if cell == None:
                            continue
                        direction = getDirTo(ship.position, cell.position, size)
                        oneStepAheadPos.append(cell.position)
                        ship.next_action = direction
                        
                    if safeCells == None or len(safeCells) == 0:
                        if ship.halite > convertCost:
                            ship.next_action = ShipAction.CONVERT
                    
                elif enemies != None or len(enemies) > 0:
                    if ship.id in goToCell.keys():
                        del goToCell[ship.id]
                        del goToPos[ship.id]
                    safeCells = getSafeRunAwayCells(ship, enemies, oneStepAheadPos)
                    if safeCells != None and len(safeCells) > 0:
                        direction = getDirTo(ship.position, safeCells[0].position, size)
                        oneStepAheadPos.append(safeCells[0].position)
                        ship.next_action = direction
                    if safeCells == None or len(safeCells) == 0:
                        if ship.halite > convertCost:
                            ship.next_action = ShipAction.CONVERT
                
            elif ((ship.halite > 700 and board.step>=170 )or ship.halite>450) and len(me.shipyards) > 0:
                    
                if ship.id in goToCell.keys():
                    del goToCell[ship.id]
                    del goToPos[ship.id]
                        
                shipyardCell, steps = nearestShipyard(ship, me.shipyards, size)
                    
                if steps > 15:
                    ship.next_action = ShipAction.CONVERT
                    continue
                        
                cell = getSafestAndQuickestPath(shipyardCell, safeCells, size)
                if cell == None:
                    continue
                direction = getDirTo(ship.position, cell.position, size)
                oneStepAheadPos.append(cell.position)
                ship.next_action = direction
                    
            elif ship.cell.shipyard != None:
                highestHaliteCell = getNearbyHighestHaliteCell(board, ship, size, occupiedCells, oneStepAheadPos)
                    
                if highestHaliteCell != None: 
                    cell = getSafestAndQuickestPath(highestHaliteCell, safeCells, size)
                    if cell == None:
                        continue
                    goToCell[ship.id] = highestHaliteCell
                    goToPos[ship.id] = highestHaliteCell.position
                    direction = getDirTo(ship.position, cell.position, size)
                    oneStepAheadPos.append(cell.position)
                    ship.next_action = direction
                        
                elif highestHaliteCell == None:
                    safestHighestHaliteCellAround = cellWithHighestHalite(safeCells, oneStepAheadPos)
                    direction = getDirTo(ship.position, safestHighestHaliteCellAround.position, size)
                    oneStepAheadPos.append(safestHighestHaliteCellAround.position)
                    ship.next_action = direction
                        
            elif ((ship.cell.halite < 50 and board.step<250) or(ship.cell.halite < 5 and board.step>350)or(ship.cell.halite < 15 and (board.step>=250 and board.step<=350))) and ship.id not in goToCell.keys():
                safestHighestHaliteCellAround = cellWithHighestHalite(safeCells, oneStepAheadPos)
                    
                if safestHighestHaliteCellAround != None:
                    direction = getDirTo(ship.position, safestHighestHaliteCellAround.position, size)
                    oneStepAheadPos.append(safestHighestHaliteCellAround.position)
                    ship.next_action = direction
                        
                elif safestHighestHaliteCellAround == None:
                    highestHaliteCell = getNearbyHighestHaliteCell(board, ship, size, occupiedCells, oneStepAheadPos)
                        
                    if highestHaliteCell != None: 
                        cell = getSafestAndQuickestPath(highestHaliteCell, safeCells, size)
                        if cell == None:
                            continue
                        goToCell[ship.id] = highestHaliteCell
                        goToPos[ship.id] = highestHaliteCell.position
                        direction = getDirTo(ship.position, cell.position, size)
                        oneStepAheadPos.append(cell.position)
                        ship.next_action = direction
                        
            elif ship.id in goToCell.keys():
                cellSafe = stateOfCell(goToCell[ship.id])
                    
                if cellSafe == True:
                    cell = getSafestAndQuickestPath(goToCell[ship.id], safeCells, size)
                    if cell == None:
                        continue
                    direction = getDirTo(ship.position, cell.position, size)
                    oneStepAheadPos.append(cell.position)
                    ship.next_action = direction
                        
                elif cellSafe == False:
                    del goToCell[ship.id]
                    del goToPos[ship.id]
                        
                    safestHighestHaliteCellAround = cellWithHighestHalite(safeCells, oneStepAheadPos)
                        
                    if safestHighestHaliteCellAround != None:
                        direction = getDirTo(ship.position, safestHighestHaliteCellAround.position, size)
                        oneStepAheadPos.append(safestHighestHaliteCellAround.position)
                        ship.next_action = direction
                            
                    elif safestHighestHaliteCellAround == None:
                        highestHaliteCell = getNearbyHighestHaliteCell(board, ship, size, occupiedCells, oneStepAheadPos)
                        
                        if highestHaliteCell != None: 
                            cell = getSafestAndQuickestPath(highestHaliteCell, safeCells, size)
                            if cell == None:
                                continue
                            goToCell[ship.id] = highestHaliteCell
                            goToPos[ship.id] = highestHaliteCell.position
                            direction = getDirTo(ship.position, cell.position, size)
                            oneStepAheadPos.append(cell.position)
                            ship.next_action = direction
    
    oneStepBehindCells = occupiedCells
                    
    return me.next_actions